# Environment

In [ ]:
# whether to use Spark when performing grid search for hyperparameters
use_spark = False
# the number of grid search processes to spawn if NOT using Spark
n_jobs = 4

# Imports

In [ ]:
import numpy as np
np.random.seed(1)

In [ ]:
import os
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
if not os.path.exists('results'):
    os.makedirs('results')

# $k$NN 

In [ ]:
clf = KNeighborsRegressor()
clf

In [ ]:
train = pd.read_csv('../feature_engineering/train.gz')
X_train = train[train.columns[:-1]].values
y_train = train[train.columns[-1]].values

In [ ]:
test = pd.read_csv('../feature_engineering/test.gz')
X_test = test[test.columns[:-1]].values
y_test = test[test.columns[-1]].values

## Grid Search

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=1)
cv

In [ ]:
params = {
    "n_neighbors": np.arange(1, 100, 4),
    "metric": ["euclidean", "manhattan", "chebyshev", "hamming", "canberra", "braycurtis"],
    "weights": ["uniform", "distance"],
}

In [ ]:
# setup the keyword arguments for grid search
kwargs = {
    'estimator': clf, 
    'param_grid': params, 
    'cv': cv,
}
# setup the grid based on whether spark is being used
if use_spark:
    from spark_sklearn import GridSearchCV
    grid = GridSearchCV(sc, **kwargs)
else:
    from sklearn.model_selection import GridSearchCV
    grid = GridSearchCV(**kwargs, n_jobs=n_jobs)

grid

In [ ]:
_ = grid.fit(X_train, y_train)

In [ ]:
best = pd.Series(grid.best_estimator_.get_params())
best.to_csv('results/knn-params.csv')
best

In [ ]:
results = grid.cv_results_
df = pd.DataFrame(results)
df.to_csv('results/knn-grid.csv')
df

# Testing

In [ ]:
clf = KNeighborsRegressor(**grid.best_estimator_.get_params(), n_jobs=n_jobs)
clf

In [ ]:
clf.fit(X_train, y_train)

## Training Accuracy

In [ ]:
train_pred = clf.predict(X_train)

In [ ]:
train_err = mean_squared_error(y_train, train_pred)
train_err

In [ ]:
train_df = pd.DataFrame([y_train, train_pred], index=['Truth', 'Predictions']).T

In [ ]:
ax = train_df.plot(figsize=(16, 4))
ax.set_ylabel('$\MWHr')
ax.set_xlabel('Time Step')
plt.savefig('results/knn-train.pdf')
plt.savefig('results/knn-train.svg')

## Testing Accuracy

In [ ]:
test_pred = clf.predict(X_test)

In [ ]:
test_err = mean_squared_error(y_test, test_pred)
test_err

In [ ]:
test_df = pd.DataFrame([y_test, test_pred], index=['Truth', 'Predictions']).T

In [ ]:
ax = test_df.plot(figsize=(16, 4))
ax.set_ylabel('$\MWHr')
ax.set_xlabel('Time Step')
plt.savefig('results/knn-test.pdf')
plt.savefig('results/knn-test.svg')

## Results

In [ ]:
results = pd.Series([train_err, test_err], index=['Train', 'Test'])
results

In [ ]:
results.to_csv('results/knn-mse.csv')